# Agent Forms and Operatives Usage Patterns

This notebook demonstrates common usage patterns for the Lion OS agent decision system.

In [ ]:
from lion.core.forms.agent_forms import (
    SelectiveAgentForm,
    CreativeAgentForm, 
    PlanningAgentForm,
    CompositeAgentForm,
    AgentDecisionReport
)
from lion.core.operatives.agent_operatives import (
    SelectAgentOperative,
    BrainstormAgentOperative,
    PlanAgentOperative,
    AgentOperativeFactory,
    AgentOperativeComposer
)

## 1. Basic Selection Pattern

In [ ]:
# Using SelectiveAgentForm directly
choices = ["option1", "option2", "option3"]
form = SelectiveAgentForm(
    choices=choices,
    max_selections=2,
    context={"goal": "Select the best two options"}
)

result = await form.execute()
print(f"Selected: {result.selected}")

# Using SelectAgentOperative
operative = SelectAgentOperative(
    choices=choices,
    max_selections=2
)
operative.extend_context(goal="Select the best two options")

result = await operative.execute()
print(f"Selected via operative: {result.selected}")

## 2. Creative Brainstorming Pattern

In [ ]:
# Using CreativeAgentForm with filters
form = CreativeAgentForm(
    num_ideas=5,
    auto_execute=True,
    idea_filters=["uniqueness", "relevance"],
    context={"topic": "AI safety measures"}
)

ideas = await form.execute()
print(f"Generated {len(ideas)} unique and relevant ideas")

# Using BrainstormAgentOperative
operative = BrainstormAgentOperative(
    num_ideas=5,
    auto_execute=True
)
operative.extend_context(topic="AI safety measures")

ideas = await operative.execute()
print(f"Generated ideas via operative: {len(ideas)}")

## 3. Planning Pattern with Dependencies

In [ ]:
# Using PlanningAgentForm with dependencies
form = PlanningAgentForm(
    num_steps=4,
    sequential=False,
    dependencies={
        "step2": ["step1"],
        "step3": ["step1", "step2"],
        "step4": ["step3"]
    },
    context={"objective": "Build ML pipeline"}
)

plan_results = await form.execute()
print("Plan execution results:")
for result in plan_results:
    print(f"Step {result['step']['id']}: {result['status']}")

## 4. Composite Decision Pattern

In [ ]:
# Create a multi-stage decision process
selector = SelectiveAgentForm(
    choices=["ML", "CV", "NLP"],
    max_selections=1
)

brainstormer = CreativeAgentForm(
    num_ideas=3,
    idea_filters=["uniqueness"]
)

planner = PlanningAgentForm(
    num_steps=3,
    sequential=True
)

composite = CompositeAgentForm(
    steps=[selector, brainstormer, planner]
)

results = await composite.execute()
print("Composite decision results:")
for stage, result in results.items():
    print(f"{stage}: {result}")

## 5. Factory and Composer Pattern

In [ ]:
# Load operatives from config
factory = AgentOperativeFactory.load_config("operatives.yaml")

# Create workflow
composer = AgentOperativeComposer(factory)

composer.add_step(
    "select_domain",
    "select_options",
    context={"choices": ["ML", "CV", "NLP"]}
)

composer.add_step(
    "generate_ideas",
    "brainstorm",
    depends_on=["select_domain"],
    context_map={
        "domain": "select_domain.selected[0]"
    }
)

composer.add_step(
    "create_plan",
    "plan",
    depends_on=["generate_ideas"],
    context_map={
        "ideas": "generate_ideas.instruct_models"
    }
)

workflow_results = await composer.execute()
print("Workflow execution results:")
for step, result in workflow_results.items():
    print(f"{step}: {result}")

## 6. Decision Reporting Pattern

In [ ]:
# Track decision outcomes
report = AgentDecisionReport()

# Add results with metadata
report.add_decision_result(
    composite,
    success=True,
    metadata={"execution_time": 1.5}
)

# Get statistics
stats = report.get_stage_stats()
print("Decision statistics:")
for stage, metrics in stats.items():
    print(f"\n{stage}:")
    print(f"Success rate: {metrics['success_rate']:.2%}")
    print(f"Total attempts: {metrics['total_attempts']}")
    if metrics['common_failures']:
        print("Common failures:", metrics['common_failures'])